<div align="center">

# **Limpieza**

</div>

## Librerias

In [48]:
library(dplyr)
library(tidyr)
source("../src/data/dividir_polizas_por_anio.R")
source("../src/data/ajustar_inflacion.R")
source("../src/data/Freedman_Diaconis.R")

## Data

In [49]:
df_input <- read.csv("../data/input/Muestra_Siniestros_4.csv")
dim(df_input)
glimpse(df_input)

[1] 5000   18

Rows: 5,000
Columns: 18
$ Amparo          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PERDI…
$ Amp             <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PPD",…
$ SumaDePagos     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 263295…
$ Modelo          <int> 2013, 2010, 2012, 2008, 2008, 2007, 2012, 2007, 2010, …
$ Color           <chr> "ROJO", "PLATA", "PLATA", "BLANCO", "VINO TINTO", "PLA…
$ Carroceria      <chr> "SEDAN", "HATCHBACK", "SEDAN", "SEDAN", "SEDAN", "HATC…
$ MARCA           <chr> "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI",…
$ Referencia1     <chr> "VELOSTER", "ATOS [2]", "ACCENT i25", "ATOS [2]", "ACC…
$ Referencia2     <chr> "COUPE", "PRIME", "1.4L", "PRIME", "GLS", "SANTRO", "G…
$ CLASE_FASECOLDA <chr> "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL TAXI…
$ TIPO_VEHICULO   <chr> "Livianos", "Livianos", "Livianos", "Livianos", "Livia…
$ SERVICIO        <chr> "Particular", "Particular", "Particular", "Particular"…
$ Sexo_Aseg     

In [50]:
unique(df_input$MARCA)

[1] "HYUNDAI"

In [51]:
nrow(df_input)

[1] 5000

In [52]:
colSums(is.na(df_input))

Amparo             Amp     SumaDePagos          Modelo           Color 
           4320            4320            4320               0               0 
     Carroceria           MARCA     Referencia1     Referencia2 CLASE_FASECOLDA 
              0               0               0               0               0 
  TIPO_VEHICULO        SERVICIO       Sexo_Aseg            Edad           Desde 
              0               0             625               0               0 
          Hasta    Vr_Comercial            Pago 
              0               0               0

## Preparacion de los datos

Nombres de variables

In [53]:
df_input$Color <- gsub("CHAMPA�A", "CHAMPAÑA", df_input$Color)
df_input$Amparo <- gsub("DA�OS", "DAÑOS", df_input$Amparo)

Nulos en Amp

In [54]:
df_input <- df_input %>%
  mutate(
    # Convertir NA a "No aplica"
    Amparo = ifelse(is.na(Amparo), "No aplica", Amparo),
    Amp = ifelse(is.na(Amp), "No aplica", Amp)
    )

Nulos en SumaDePagos

In [55]:
head(sort(unique(df_input$SumaDePagos)))

[1]     0 26248 60000 69257 70000 80000

In [56]:
pago_minimo <- 100000
df_input <- df_input %>% mutate(
    SumaDePagos = case_when(
      is.na(SumaDePagos) ~ 0,
      SumaDePagos < pago_minimo ~ 0,
      TRUE ~ SumaDePagos)
)

df_input <- df_input %>% mutate(
  Accidentado = ifelse(SumaDePagos >= pago_minimo, 1, 0)
)

El pago minimo se escogio como el valor de un espejo en 2015

Nulos en genero

In [57]:
df_input <- df_input %>% mutate(
    Sexo_Aseg = ifelse(is.na(Sexo_Aseg), "No aplica", Sexo_Aseg)
)

In [58]:
nrow(df_input[(df_input$Sexo_Aseg == "No aplica"), ])/nrow(df_input)

[1] 0.125

In [59]:
df_input <- df_input[(df_input$Sexo_Aseg != "No aplica"), ]

Segmentando por año

In [60]:
df_input <- dividir_polizas_por_anio(df_input)

Calculo de exposicion

In [61]:
df_input$exposicion <- as.numeric(df_input$Hasta - df_input$Desde) + 1

Ajustando a inflacion de 2015

In [62]:
max(df_input$Hasta)

[1] "2015-02-21"

In [63]:
df_input <- ajustar_inflacion_2015(df_input)

Llevamos todo a los mismo valores presentes ajustando la inflacion.

Variables que sobran

In [64]:
df_input[c("Amp", "MARCA")] <- NULL

Ajustando valor comercial

In [65]:
vr_minimo <- 4000000
df_input <- df_input[(df_input$Vr_Comercial > vr_minimo), ]

El valor comercial minimo se escogio como el valor de un carro hyundai Excel en 2015

Ajustando edad

In [66]:
edad_minima <- 18
edad_maxima <- 90

df_input <- df_input[(df_input$Edad >= edad_minima & df_input$Edad <= edad_maxima), ]

Resultados

In [67]:
colSums(is.na(df_input))

Amparo     SumaDePagos          Modelo           Color      Carroceria 
              0               0               0               0               0 
    Referencia1     Referencia2 CLASE_FASECOLDA   TIPO_VEHICULO        SERVICIO 
              0               0               0               0               0 
      Sexo_Aseg            Edad           Desde           Hasta    Vr_Comercial 
              0               0               0               0               0 
           Pago     Accidentado      exposicion 
              0               0               0

In [68]:
summary(df_input)

    Amparo           SumaDePagos           Modelo        Color          
 Length:6750        Min.   :       0   Min.   :1993   Length:6750       
 Class :character   1st Qu.:       0   1st Qu.:2008   Class :character  
 Mode  :character   Median :       0   Median :2010   Mode  :character  
                    Mean   :  281823   Mean   :2009                     
                    3rd Qu.:       0   3rd Qu.:2012                     
                    Max.   :35206209   Max.   :2013                     
  Carroceria        Referencia1        Referencia2        CLASE_FASECOLDA   
 Length:6750        Length:6750        Length:6750        Length:6750       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                           

In [69]:
# Aplicar la función Freedman_Diaconis para crear grupos de edad
resultado_grupos <- Freedman_Diaconis(df_input$Edad, n_categories = 3)

# Mostrar información sobre los grupos creados
cat("Grupos de edad creados:\n")
print(resultado_grupos$category_labels)

cat("\nBreaks utilizados:\n")
print(resultado_grupos$breaks)

cat("\nResumen de grupos:\n")
print(resultado_grupos$summary)

Grupos de edad creados:
[1] "Edad_19_41" "Edad_41_63" "Edad_63_85"

Breaks utilizados:
[1] 18.999 41.000 63.000 85.001

Resumen de grupos:
categories
Edad_19_41 Edad_41_63 Edad_63_85 
      2728       3448        574 


In [70]:
# Agregar la nueva variable de grupo de edad al dataframe
df_input$Grupo_Edad <- resultado_grupos$categories

# Verificar la distribución de los grupos de edad
table(df_input$Grupo_Edad)

# Crear etiquetas más legibles para los grupos de edad
df_input <- df_input %>%
  mutate(Grupo_Edad = case_when(
    Grupo_Edad == resultado_grupos$category_labels[1] ~ "19-41",
    Grupo_Edad == resultado_grupos$category_labels[2] ~ "41-63", 
    Grupo_Edad == resultado_grupos$category_labels[3] ~ "63-85",
    TRUE ~ as.character(Grupo_Edad)
  ))

# Verificar la nueva distribución
table(df_input$Grupo_Edad)


Edad_19_41 Edad_41_63 Edad_63_85 
      2728       3448        574 


19-41 41-63 63-85 
 2728  3448   574 

## Creación de grupos de edad

Exportando los resultados

In [71]:
write.csv(df_input, "../data/processed/datos_limpios.csv", row.names = FALSE)